# Snowpark Basics HoL Part 3 - Writing to Tables

## 3.1 Setup

### Imports
These imports are from our local Python environment, snowparkbasics.

In [ ]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

import sys
import json
import pandas as pd
import numpy as np

# Make sure we do not get line breaks when doing show on wide dataframes
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Create Snowpark Session

In [ ]:
with open('creds.json') as f:
    connection_parameters = json.load(f)

In [ ]:
session = Session.builder.configs(connection_parameters).create()
print(f"Current Database and schema: {session.get_fully_qualified_current_schema()}")
print(f"Current Warehouse: {session.get_current_warehouse()}")

### Create Stage
This will be used below.

In [ ]:
session.sql('CREATE OR REPLACE STAGE TRUCK_STAGE').collect()

## 3.2 Loading Tables

### Loading Data into Snowflake
We can load data into Snowflake, including running a PUT operation from the Snowpark client.
<br> CSV data is a little more complex in that a structure may need to be explicitly defined.

In [ ]:
put_result = session.file.put('data/truck.csv', "@raw_pos.truck_stage", overwrite=True)
put_result

In [ ]:
session.sql('LIST @TRUCK_STAGE').collect()

In [ ]:
# We could use sesson.sql
# session.sql('COPY INTO TRUCK1 from @truck_stage file_format=(skip_header=1) on_error=continue').collect()

# user_schema is used to read from CSV files. For other files it's not needed.
truck_schema = T.StructType([T.StructField("TRUCK_ID", T.IntegerType()),
    T.StructField("MENU_TYPE_ID", T.IntegerType()),
    T.StructField("PRIMARY_CITY", T.StringType()),
    T.StructField("REGION", T.StringType()),
    T.StructField("ISO_REGION", T.StringType()),
    T.StructField("COUNTRY", T.StringType()),
    T.StructField("ISO_COUNTRY_CODE", T.StringType()),
    T.StructField("FRANCHISE_FLAG", T.IntegerType()),
    T.StructField("YEAR", T.IntegerType()),
    T.StructField("MAKE", T.StringType()),
    T.StructField("MODEL", T.StringType()),
    T.StructField("EV_FLAG", T.IntegerType()),
    T.StructField("FRANCHISE_ID", T.IntegerType()),
    T.StructField("TRUCK_OPENING_DATE", T.DateType())])

# We wtill start with a new table. Snowpark copy_into_table will create a table if necessary (but can also copy into an existing populated table).
drop_result = session.sql("drop table if exists TRUCK1").collect()

# Use the DataFrameReader (session.read below) to read from CSV files.
truck_df = session.read.schema(truck_schema).csv("@truck_stage")

csv_file_format_options = {"FIELD_OPTIONALLY_ENCLOSED_BY": "'\"'", "skip_header": 1}
copied_into_result = truck_df.copy_into_table("TRUCK1", format_type_options=csv_file_format_options)
copied_into_result

In [ ]:
session.table("TRUCK1").to_pandas()

The new infer schema capability can be used but at present requires that the table be loaded via INSERT SELECT rather than COPY

In [ ]:
drop_result = session.sql("drop table if exists TRUCK2").collect()
infer_df = session.read.option("INFER_SCHEMA", True).option("PARSE_HEADER", True).csv("@truck_stage/truck.csv")
infer_df.show()
infer_df.write.mode("overwrite").saveAsTable("TRUCK2")

In [ ]:
session.table("TRUCK2").to_pandas()

## 3.3 Writing to Tables
We have already seen ways of creating new tables from data. But what if we want to insert, update, or delete existing tables?
To keep it simple let's start with a new table TRUCKUS1 just holding US trucks.

In [ ]:
truckus1_df = session.table("TRUCK1").filter(F.col("ISO_COUNTRY_CODE") == "US")
truckus1_df.write.mode("overwrite").saveAsTable("TRUCKUS1")
session.table("TRUCKUS1").to_pandas()

### Copying a Table
The following will use INSERT...SELECT. Currently to CLONE a table requires session.sql.

In [ ]:
truckus2_df = session.table("TRUCKUS1")
truckus2_df.write.mode("overwrite").saveAsTable("TRUCKUS2")
session.table("TRUCKUS2").to_pandas()

Let's create a new set of data by adding 1000 to the original truck ids.
<br>(We could also use withColumn, reusing the column name, to update TRUCK_ID but it moves the column to the end...)

In [ ]:
truck1000_df = truckus2_df.select((F.col("TRUCK_ID") + 1000).alias("TRUCK_ID"), F.col("MENU_TYPE_ID"), F.col("PRIMARY_CITY"), F.col("REGION"),
        F.col("ISO_REGION"), F.col("COUNTRY"), F.col("ISO_COUNTRY_CODE"), F.col("FRANCHISE_FLAG"), F.col("YEAR"), F.col("MAKE"), 
        F.col("MODEL"), F.col("EV_FLAG"), F.col("FRANCHISE_ID"), F.col("TRUCK_OPENING_DATE" ))
truck1000_df.to_pandas()

### Inserting to a Table
Currently there isn't an explicit insert API call. This call will insert an additional 75 rows to TRUCKUS1.

In [ ]:
truck1000_df.write.mode('append').saveAsTable("TRUCKUS1")
session.table("TRUCKUS1").to_pandas()

## 3.4 Updating, Deleting and Merging
Currently updating, deleting and merging data in or from a table can be done via snowpark.Table.
<br>Note that the new Python 'core' API is likely to introduce new calls for tables in the next few months.

### Updating a Table
Let's update the TRUCKUS2 table.

In [ ]:
t=session.table("TRUCKUS2")
t.to_pandas()

We can list columns to update and a condition in the **Table.update()** method:

In [ ]:
t.update({"FRANCHISE_FLAG" : 99}, F.col("ISO_REGION")=="CA")

In [ ]:
session.table("TRUCKUS2").filter(F.col("ISO_REGION") =="CA").to_pandas()

We can also update one table based on values in another table - in this case TRUCKUS1 based on values in TRUCKUS2.
<br>This should update only the intiial set of TRUCKUS1 rows which match the TRUCKUS2 rows.

In [ ]:
source_df = session.table("TRUCKUS2") 
target_df = session.table("TRUCKUS1")
target_df.update({"FRANCHISE_FLAG" : source_df["FRANCHISE_FLAG"]}, F.col("TRUCK_ID") == source_df["TRUCK_ID"],source_df)

In [ ]:
session.table("TRUCKUS1").filter(F.col("ISO_REGION") =="CA").show(50)

### Deleting from a Table
We can list columns and a condition in the **Table.delete()** method. Let's delete from TRUCK3 based on a condtition.
<br>We delete based on the FRANCHISE_FLAG, but that is the same as all the rows from CA.

In [ ]:
t=session.table("TRUCKUS2")
t.delete(F.col("FRANCHISE_FLAG") == 99)

In [ ]:
session.table("TRUCKUS2").filter(F.col("ISO_REGION") =="CA").to_pandas()

We can also delete one table based on values in another table.  Here we use the remaining 60 TRUCKUS2 rows to provide a set of keys to delete from the TRUCKUS1 table, leaving the additional rows with plus-1000 kys and the original CA rows.

In [ ]:
source_df = session.table("TRUCKUS2")
target_df = session.table("TRUCKUS1")
target_df.delete(F.col("TRUCK_ID") == source_df["TRUCK_ID"],source_df)

In [ ]:
session.table("TRUCKUS1").show(100)

In [ ]:
session.table("TRUCKUS2").show(100)

### Simple Merge Example
Merging can get complicated to follow whether in SQL or any other language!  
Here is a much simpler example using a table we create inline.

In [ ]:
session.create_dataframe([(10, "old"), (10, "unknown"), (11, "old")],
      schema=["key", "value"]
   ).write.save_as_table("my_table", mode="overwrite", table_type="temporary")

target = session.table("my_table")

source = session.create_dataframe([(10, "new"), (12, "new"), (13, "new")],
   schema=["key", "value"])

target.merge(source,
   (target.key == source.key) & (target.value == "unknown"),
   [F.when_matched().update({"value": source["value"]}),
   F.when_not_matched().insert({"key": source["key"],"value": source["value"]})])

session.table("my_table").sort(F.col('KEY')).show()

## 3.X YOUR TURN!
Here is the challenge:
<br>Using the infer capability, create a new ONETRUCKHEADER table by loading from the header.csv file.
<br>Then update the new table to set SHIFT_ID to 99.
<br>Create another table TWOTRUCKHEADER by copying data from ORDER_HEADER (not ONETRUCKHEADER) with TRUCK_ID 122 or 123.
<br>Update TWOTRUCKHEADER setting the SHIFT_ID to the one from ONETRUCKHEADER for the same ORDER_ID.
<br>Finally check the update worked e.g. count TWOTRUCKHEADER rows grouped by TRUCK_ID and SHIFT_ID.

### Create a stage and PUT the file

In [ ]:
session.sql('CREATE OR REPLACE STAGE ONETRUCK_STAGE').collect()

In [ ]:
put_result = session.file.put('data/header.csv', "@raw_pos.onetruck_stage", overwrite=True)
put_result

### Now use the infer capability to create and then load the ONETRUCKHEADER table

In [ ]:
infer_df = session.read.option("INFER_SCHEMA", True).option("PARSE_HEADER", True).csv("@onetruck_stage/header.csv")
infer_df.show()
infer_df.write.mode("overwrite").saveAsTable("ONETRUCKHEADER")

### Update ONETRUCKHEADER to set the SHIFT_ID to 99

In [ ]:
onetruck=session.table("ONETRUCKHEADER")
onetruck.update({"SHIFT_ID" : 99})
session.table("ONETRUCKHEADER").show()

### Create a TWOTRUCKHEADER table for TRUCK_ID 122 or 123

In [ ]:
twotruck_df = session.table("ORDER_HEADER").filter(F.col("TRUCK_ID").in_(122, 123))
twotruck_df.show()
twotruck_df.write.mode("overwrite").saveAsTable("TWOTRUCKHEADER")

### Update TWOTRUCKHEADER SHIFT_IDs based on ONETRUCKHEADER

In [ ]:
source_df = session.table("ONETRUCKHEADER") 
target_df = session.table("TWOTRUCKHEADER")
update_result = target_df.update({"SHIFT_ID" : source_df["SHIFT_ID"]}, F.col("ORDER_ID") == source_df["ORDER_ID"],source_df)
update_result

### Count rows in TWOTRUCKHEADER

In [ ]:
target_df = session.table("TWOTRUCKHEADER")
target_df.groupBy(['TRUCK_ID','SHIFT_ID']).agg(
             [F.count('*').alias('COUNT')]).show()

In [ ]:
session.close()